# Generate Bidirectional best hits for two assemblies of the *Yersinia pestis* CO92 genome

In [1]:
!ls *CO92*.faa

Y_pestis_CO92.faa      Y_pestis_CO92_2014.faa


In [4]:
%%bash
ORG1=Y_pestis_CO92_2014
ORG2=Y_pestis_CO92
for i in `seq 1 4`
do
    exonerate   --query $ORG1.faa \
                --target $ORG2.faa \
                --querychunkid $i \
                --querychunktotal 4 \
                --bestn 1 \
                --ryo "%qi\t%ti\t%ps\n" \
                --showvulgar no \
                --verbose 0 \
                --showalignment no > $ORG1.to.$ORG2.$i.tsv &
done
for i in `seq 1 4`
do
    exonerate   --target $ORG1.faa \
                --query $ORG2.faa \
                --querychunkid $i \
                --querychunktotal 4 \
                --bestn 1 \
                --ryo "%qi\t%ti\t%ps\n" \
                --showvulgar no \
                --verbose 0 \
                --showalignment no > $ORG2.to.$ORG1.$i.tsv &
done

##  Concatenate output chunks into one output file for each mapping

In [5]:
%%bash
ORG1=Y_pestis_CO92_2014
ORG2=Y_pestis_CO92

echo "$ORG1	$ORG2	Similarity_$ORG1.to.$ORG2" > $ORG1.to.$ORG2.tab
echo "$ORG2	$ORG1	Similarity_$ORG2.to.$ORG1" > $ORG2.to.$ORG1.tab
for i in `seq 1 4`
do
    cat $ORG1.to.$ORG2.$i.tsv >> $ORG1.to.$ORG2.tab
    cat $ORG2.to.$ORG1.$i.tsv >> $ORG2.to.$ORG1.tab
done

## Read the mappings in to a pandas dataframe

In [6]:
import pandas as pd
org1 = 'Y_pestis_CO92_2014'
org2 = 'Y_pestis_CO92'
org1_to_org2 = pd.read_table('{}.to.{}.tab'.format(org1, org2))
org2_to_org1 = pd.read_table('{}.to.{}.tab'.format(org2,org1))
display(org1_to_org2.sort_values(by='Similarity_{}.to.{}'.format(org1,org2),ascending=False).head(10))
display(org2_to_org1.sort_values(by='Similarity_{}.to.{}'.format(org2,org1),ascending=False).head(10))

,Y_pestis_CO92_2014,Y_pestis_CO92,Similarity_Y_pestis_CO92_2014.to.Y_pestis_CO92
0,KGA47888.1,gnl|YPES214092|GKDD-2986-MONOMER,100.0
6402,KGA54032.1,gnl|YPES214092|GKDD-1872-MONOMER,100.0
6396,KGA54032.1,gnl|YPES214092|GKDD-1982-MONOMER,100.0
6397,KGA54032.1,gnl|YPES214092|GKDD-4116-MONOMER,100.0
6398,KGA54032.1,gnl|YPES214092|GKDD-3833-MONOMER,100.0
6399,KGA54032.1,gnl|YPES214092|GKDD-1032-MONOMER,100.0
6400,KGA54032.1,gnl|YPES214092|GKDD-2505-MONOMER,100.0
6401,KGA54032.1,gnl|YPES214092|GKDD-2063-MONOMER,100.0
6403,KGA54032.1,gnl|YPES214092|GKDD-3034-MONOMER,100.0
6394,KGA54032.1,gnl|YPES214092|GKDD-2583-MONOMER,100.0


,Y_pestis_CO92,Y_pestis_CO92_2014,Similarity_Y_pestis_CO92.to.Y_pestis_CO92_2014
0,gnl|YPES214092|GKDD-100-MONOMER,KGA52105.1,100.0
2966,gnl|YPES214092|GKDD-3058-MONOMER,KGA52519.1,100.0
2973,gnl|YPES214092|GKDD-3077-MONOMER,KGA51993.1,100.0
2972,gnl|YPES214092|GKDD-3076-MONOMER,KGA52007.1,100.0
2971,gnl|YPES214092|GKDD-3075-MONOMER,KGA52005.1,100.0
2970,gnl|YPES214092|GKDD-3067-MONOMER,KGA52437.1,100.0
2968,gnl|YPES214092|GKDD-3063-MONOMER,KGA52377.1,100.0
2967,gnl|YPES214092|GKDD-3059-MONOMER,KGA52433.1,100.0
2965,gnl|YPES214092|GKDD-3051-MONOMER,KGA52482.1,100.0
2957,gnl|YPES214092|GKDD-3021-MONOMER,KGA54701.1,100.0


## Take the intersection of mappings to get bidirection best hits.

In [7]:
from IPython.display import display, HTML
org1 = 'Y_pestis_CO92_2014'
org2 = 'Y_pestis_CO92'
bbh  = org1_to_org2.merge(org2_to_org1, 
                 on=[org1, org2], 
                 how='inner',
                 #suffixes=('_albicans2auris','_auris2albicans'),
                 sort=True).sort_values(by='Similarity_{}.to.{}'.format(org1,org2),ascending=False)
display(HTML(bbh.to_html()))
bbh.to_csv('{}_and_{}_BBH.tab'.format(org1,org2), index=False, sep='\t')

,Y_pestis_CO92_2014,Y_pestis_CO92,Similarity_Y_pestis_CO92_2014.to.Y_pestis_CO92,Similarity_Y_pestis_CO92.to.Y_pestis_CO92_2014
0,KGA47888.1,gnl|YPES214092|GKDD-2986-MONOMER,100.00,100.00
2967,KGA54241.1,gnl|YPES214092|GKDD-2856-MONOMER,100.00,100.00
2973,KGA54247.1,gnl|YPES214092|GKDD-2869-MONOMER,100.00,100.00
2972,KGA54246.1,gnl|YPES214092|GKDD-2916-MONOMER,100.00,100.00
2971,KGA54245.1,gnl|YPES214092|GKDD-2949-MONOMER,100.00,100.00
2970,KGA54244.1,gnl|YPES214092|GKDD-2851-MONOMER,100.00,100.00
2969,KGA54243.1,gnl|YPES214092|GKDD-2880-MONOMER,100.00,100.00
2968,KGA54242.1,gnl|YPES214092|GKDD-2942-MONOMER,100.00,100.00
2966,KGA54240.1,gnl|YPES214092|GKDD-2846-MONOMER,100.00,100.00
2924,KGA54190.1,gnl|YPES214092|GKDD-3405-MONOMER,100.00,100.00
